In [11]:
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

consumerTrain = pd.read_csv("train_dataset.csv")
consumerTest = pd.read_csv("test_dataset.csv")

Id = consumerTest["用户编码"]
consumerTest = consumerTest.drop(["用户编码"],axis=1)


print("-"*50+ "train简单查看统计学信息" +"-"*50)
print(consumerTrain.info())
print("-"*50+ "test简单查看统计学信息" +"-"*50)
print(consumerTest.info())

print("-"*50 + "train查看缺失值" + "-"*50)
print(consumerTrain.isnull().sum().sort_values(ascending=False).head(10))
print("-"*50 + "test查看缺失值" + "-"*50)
print(consumerTest.isnull().sum().sort_values(ascending=False).head(10))


--------------------------------------------------train简单查看统计学信息--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 30 columns):
用户编码               50000 non-null object
用户实名制是否通过核实        50000 non-null int64
用户年龄               50000 non-null int64
是否大学生客户            50000 non-null int64
是否黑名单客户            50000 non-null int64
是否4G不健康客户          50000 non-null int64
用户网龄（月）            50000 non-null int64
用户最近一次缴费距今时长（月）    50000 non-null int64
缴费用户最近一次缴费金额（元）    50000 non-null float64
用户近6个月平均消费值（元）     50000 non-null float64
用户账单当月总费用（元）       50000 non-null float64
用户当月账户余额（元）        50000 non-null int64
缴费用户当前是否欠费缴费       50000 non-null int64
用户话费敏感度            50000 non-null int64
当月通话交往圈人数          50000 non-null int64
是否经常逛商场的人          50000 non-null int64
近三个月月均商场出现次数       50000 non-null int64
当月是否逛过福州仓山万达       50000 non-null int64
当月是否到过福州山姆会员店      50000 non-null int64
当月是否看电影  

In [12]:
def simple_features(df_data):
    df = df_data
    df["次数"] = df["当月网购类应用使用次数"]+df["当月物流快递类应用使用次数"]+df["当月金融理财类应用使用总次数"]+df["当月视频播放类应用使用次数"]+df["当月飞机类应用使用次数"]+df["当月火车类应用使用次数"]+df["当月旅游资讯类应用使用次数"]

    df["火车飞机类应用使用次数"] = df["当月飞机类应用使用次数"]+df["当月火车类应用使用次数"]

    df["是否不良"] = df["是否黑名单客户"]+df["是否4G不健康客户"]

    df["是否不良"][df["是否不良"]>1] = 1
    
    for col in ['当月金融理财类应用使用总次数','当月旅游资讯类应用使用次数']: # 这两个比较积极向上一点
        df[col + '百分比'] = df[col].values / df['次数'].values 
    
    
    df['当月通话人均话费'] = df['用户账单当月总费用（元）'].values / (df['当月通话交往圈人数'].values + 1)
    df['上个月费用'] = df['用户当月账户余额（元）'].values + df['用户账单当月总费用（元）'].values
     
    df['用户上网年龄'] = df['用户年龄'] - df['用户网龄（月）']
    df['用户上网年龄百分比'] = df['用户网龄（月）'] / (df['用户年龄'] + 1)
     
    df['近似总消费'] = df['用户近6个月平均消费值（元）'] / 6 * df['用户网龄（月）']
    
    df = df.fillna(0)
    return df

# pd.set_option('display.max_rows',1000)   # 具体的行数或列数可自行设置
# pd.set_option('display.max_columns',1000)

In [13]:
consumerTrain = simple_features(consumerTrain)
consumerTest = simple_features(consumerTest)


In [14]:
print("-"*50+ "train简单查看统计学信息" +"-"*50)
print(consumerTrain.info())
print("-"*50+ "test简单查看统计学信息" +"-"*50)
print(consumerTest.info())

--------------------------------------------------train简单查看统计学信息--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 40 columns):
用户编码                 50000 non-null object
用户实名制是否通过核实          50000 non-null int64
用户年龄                 50000 non-null int64
是否大学生客户              50000 non-null int64
是否黑名单客户              50000 non-null int64
是否4G不健康客户            50000 non-null int64
用户网龄（月）              50000 non-null int64
用户最近一次缴费距今时长（月）      50000 non-null int64
缴费用户最近一次缴费金额（元）      50000 non-null float64
用户近6个月平均消费值（元）       50000 non-null float64
用户账单当月总费用（元）         50000 non-null float64
用户当月账户余额（元）          50000 non-null int64
缴费用户当前是否欠费缴费         50000 non-null int64
用户话费敏感度              50000 non-null int64
当月通话交往圈人数            50000 non-null int64
是否经常逛商场的人            50000 non-null int64
近三个月月均商场出现次数         50000 non-null int64
当月是否逛过福州仓山万达         50000 non-null int64
当月是否到过福州山姆会员店

In [15]:
# from sklearn import preprocessing

# def min_max_scaler(df_data):
#     df = df_data
#     df['用户年龄'] = preprocessing.MinMaxScaler(feature_range=(0,10)).fit_transform(np.array(df['用户年龄']).reshape(-1,1))
#     df['用户网龄（月）'] = preprocessing.MinMaxScaler(feature_range=(0,10)).fit_transform(np.array(df['用户网龄（月）']).reshape(-1,1))
#     df['缴费用户最近一次缴费金额（元）'] = preprocessing.MinMaxScaler(feature_range=(0,10)).fit_transform(np.array(df['缴费用户最近一次缴费金额（元）']).reshape(-1,1))
#     df['用户近6个月平均消费值（元）'] = preprocessing.MinMaxScaler(feature_range=(0,10)).fit_transform(np.array(df['用户近6个月平均消费值（元）']).reshape(-1,1))
#     df['用户账单当月总费用（元）'] = preprocessing.MinMaxScaler(feature_range=(0,10)).fit_transform(np.array(df['用户账单当月总费用（元）']).reshape(-1,1))
#     df['用户当月账户余额（元）'] = preprocessing.MinMaxScaler(feature_range=(0,10)).fit_transform(np.array(df['用户当月账户余额（元）']).reshape(-1,1))
#     df['当月通话人均话费'] = preprocessing.MinMaxScaler(feature_range=(0,10)).fit_transform(np.array(df['当月通话人均话费']).reshape(-1,1))
#     df['上个月费用'] = preprocessing.MinMaxScaler(feature_range=(0,10)).fit_transform(np.array(df['上个月费用']).reshape(-1,1))
#     df['近似总消费'] = preprocessing.MinMaxScaler(feature_range=(0,10)).fit_transform(np.array(df['近似总消费']).reshape(-1,1))
#     return df
    

In [16]:
# # pd.set_option('display.max_rows',500)   # 具体的行数或列数可自行设置
# # print(consumerTrain)
# consumerTrain = min_max_scaler(consumerTrain)
# consumerTest = min_max_scaler(consumerTest)


In [17]:
from lightgbm import LGBMRegressor 
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor


In [18]:
from sklearn.model_selection import train_test_split
from sklearn import model_selection

x = consumerTrain.drop(["信用分","用户编码"],axis=1)
y = consumerTrain.信用分
x_train,x_test, y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=10)


In [19]:
from sklearn.model_selection import KFold

SEED = 1314
NFOLDS = 7
kf = KFold(n_splits = NFOLDS, random_state=SEED, shuffle=False)

ntrain = x_train.shape[0]
ntest = consumerTest.shape[0]
print('ntrain: '+str(ntrain) +'   '+ 'ntest: '+str(ntest))

def get_out_fold(clf, x_train, y_train, consumerTest):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    for i,(train_index, test_index) in enumerate(kf.split(x_train) ):
        print('train_index:%s , test_index: %s ' %(train_index,test_index))
        x_tr = x_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        x_te = x_train.iloc[test_index]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(consumerTest)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)


SEED = 1314
NFOLDS = 7
kf = KFold(n_splits = NFOLDS, random_state=SEED, shuffle=False)

ntrain1 = x_train.shape[0]
ntest1 = x_test.shape[0]
print('ntrain1: '+str(ntrain1) +'   '+ 'ntest1: '+str(ntest1))

def get_out_fold1(clf, x_train, y_train, consumerTest):
    oof_train = np.zeros((ntrain1,))
    oof_test = np.zeros((ntest1,))
    oof_test_skf = np.empty((NFOLDS, ntest1))
    
    for i,(train_index, test_index) in enumerate(kf.split(x_train) ):
        print('train_index:%s , test_index: %s ' %(train_index,test_index))
        x_tr = x_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        x_te = x_train.iloc[test_index]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(consumerTest)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

ntrain: 40000   ntest: 50000
ntrain1: 40000   ntest1: 10000


In [20]:
print("Layer1 Start:")

xgb = XGBRegressor(max_depth=5,seed=11)

lgb = LGBMRegressor(n_estimators=1500,learning_rate=0.08,max_depth=5,bagging_fraction=0.4,
                    min_child_weight=4,num_leaves=40,alpha=0.01,random_state=11)

gb = GradientBoostingRegressor(n_estimators=1000,learning_rate=0.02,criterion='mse',random_state=11)

ada = AdaBoostRegressor(n_estimators=1000,learning_rate=0.1,random_state=11)

rf = RandomForestRegressor(n_estimators=1000,max_features=0.6,max_depth=9,min_samples_leaf=5,
                            verbose=0,oob_score=True,random_state=11)


x_train = pd.DataFrame(x_train.values)
y_train = pd.DataFrame(y_train.values)
x_test = pd.DataFrame(x_test.values)
y_test = pd.DataFrame(y_test.values)
consumerTest = pd.DataFrame(consumerTest.values)

xgb_oof_train, xgb_oof_test = get_out_fold(xgb, x_train, y_train, consumerTest) # XGBboost
# cb_oof_train, cb_oof_test = get_out_fold(cb, x_train, y_train, consumerTest) # CatBoost
gb_oof_train, gb_oof_test = get_out_fold(gb, x_train, y_train, consumerTest) # GBboost
# ada_oof_train, ada_oof_test = get_out_fold(ada, x_train, y_train, consumerTest) # Adaboost
lgb_oof_train, lgb_oof_test = get_out_fold(lgb, x_train, y_train, consumerTest) # LGBboost
rf_oof_train, rf_oof_test = get_out_fold(rf, x_train, y_train, consumerTest) # GBboost


# 为了得到维度变换后的x_test
xgb_oof_train1, xgb_oof_test1 = get_out_fold1(xgb, x_train, y_train, x_test) # XGBboost
# cb_oof_train, cb_oof_test = get_out_fold(cb, x_train, y_train, consumerTest) # CatBoost
gb_oof_train1, gb_oof_test1 = get_out_fold1(gb, x_train, y_train, x_test) # GBboost
# ada_oof_train1, ada_oof_test1 = get_out_fold1(ada, x_train, y_train, x_test) # Adaboost
lgb_oof_train1, lgb_oof_test1 = get_out_fold1(lgb, x_train, y_train, x_test) # LGBboost
rf_oof_train1, rf_oof_test1 = get_out_fold1(rf, x_train, y_train, x_test) # GBboost

print("Training is completed.")


Layer1 Start:
train_index:[ 5715  5716  5717 ... 39997 39998 39999] , test_index: [   0    1    2 ... 5712 5713 5714] 
train_index:[    0     1     2 ... 39997 39998 39999] , test_index: [ 5715  5716  5717 ... 11427 11428 11429] 
train_index:[    0     1     2 ... 39997 39998 39999] , test_index: [11430 11431 11432 ... 17141 17142 17143] 
train_index:[    0     1     2 ... 39997 39998 39999] , test_index: [17144 17145 17146 ... 22855 22856 22857] 
train_index:[    0     1     2 ... 39997 39998 39999] , test_index: [22858 22859 22860 ... 28569 28570 28571] 
train_index:[    0     1     2 ... 39997 39998 39999] , test_index: [28572 28573 28574 ... 34283 34284 34285] 
train_index:[    0     1     2 ... 34283 34284 34285] , test_index: [34286 34287 34288 ... 39997 39998 39999] 
train_index:[ 5715  5716  5717 ... 39997 39998 39999] , test_index: [   0    1    2 ... 5712 5713 5714] 
train_index:[    0     1     2 ... 39997 39998 39999] , test_index: [ 5715  5716  5717 ... 11427 11428 11429] 

In [21]:
# x_train = np.concatenate((xgb_oof_train,gb_oof_train,ada_oof_train,lgb_oof_train), axis=1)
# consumerTest = np.concatenate((xgb_oof_test,gb_oof_test,ada_oof_test,lgb_oof_test), axis=1)
# x_test = np.concatenate((xgb_oof_test1,gb_oof_test1,ada_oof_test1,lgb_oof_test1), axis=1)

x_train = np.concatenate((xgb_oof_train,gb_oof_train,lgb_oof_train,rf_oof_train), axis=1)
consumerTest = np.concatenate((xgb_oof_test,gb_oof_test,lgb_oof_test,rf_oof_test), axis=1)
x_test = np.concatenate((xgb_oof_test1,gb_oof_test1,lgb_oof_test1,rf_oof_test1), axis=1)



print("Layer1 Finished")

Layer1 Finished


In [25]:
clf_est =LGBMRegressor(random_state=1314) 

clf_param_grid = {'n_estimators': [1000],
                  'learning_rate': [0.007],
                 'max_depth':[2],
                 'bagging_fraction':[0.5],
                  'min_child_weight':[3],
                  'num_leaves':[20],
                 }

clf_grid = model_selection.GridSearchCV(clf_est, clf_param_grid, n_jobs=5, cv=2, verbose=0)
print("fitting")
clf_grid.fit(x_train, y_train)

print('BestParams: ' + str(clf_grid.best_params_))
print('Layer2 Training:')
clf = LGBMRegressor(n_estimators = clf_grid.best_estimator_.n_estimators,
                   learning_rate = clf_grid.best_estimator_.learning_rate,
                    max_depth = clf_grid.best_estimator_.max_depth,
                    bagging_fraction = clf_grid.best_estimator_.bagging_fraction,
                    min_child_weight = clf_grid.best_estimator_.min_child_weight,
                    num_leaves = clf_grid.best_estimator_.num_leaves,
                   random_state=1314)
# clf = clf_est
clf.fit(x_train, y_train)
print('Layer2 Finished')

fitting
BestParams: {'bagging_fraction': 0.5, 'learning_rate': 0.007, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 1000, 'num_leaves': 20}
Layer2 Training:
Layer2 Finished


In [26]:

predictions = clf.predict(consumerTest.astype(int))
Submission = pd.DataFrame({'id': Id, 
                           'score': predictions})
Submission["score"] = Submission["score"].astype(int)
Submission.to_csv('SubmissionStack.csv',index=False,sep=',')
print("I LOVE YOUU")

print("评分：")
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf,x_test,y_test)
print("LGB: ",scores.mean())
from sklearn import metrics

y_pred_xgb = clf.predict(x_test)
# 均方根误差
MSE = metrics.mean_squared_error(y_test,y_pred_xgb)
print(MSE)

I LOVE YOUU
评分：
LGB:  0.7922492145506052
375.7200019846286


In [27]:
print(1/(1+MSE))

0.0026544913854635284
